# HDM Train Kaggle
Created by [licyk](https://github.com/licyk)

Jupyter Notebook 仓库：[licyk/sd-webui-all-in-one](https://github.com/licyk/sd-webui-all-in-one)


## 简介
一个在 [Kaggle](https://www.kaggle.com) 部署 [HDM](https://github.com/KohakuBlueleaf/HDM) 的 Jupyter Notebook，可用于 HDM 模型的训练。[Colab](https://colab.research.google.com) 同样也可以使用。

还有这个只是写来玩的，可能有 BUG 跑不了。


## 不同运行单元的功能
该 Notebook 分为以下几个单元：

- [功能初始化](#功能初始化)
- [参数配置](#参数配置)
- [安装环境](#安装环境)
- [模型训练](#模型训练)
- [模型上传](#模型上传)

使用时请按顺序运行笔记单元。

通常情况下[功能初始化](#功能初始化)和[模型上传](#模型上传)单元的内容无需修改，其他单元包含不同功能的注释，可阅读注释获得帮助。

[参数配置](#参数配置)单元用于修改安装，训练，上传模型时的配置。

[安装](#安装)单元执行安装训练环境的命令和下载模型 / 训练集的命令，可根据需求进行修改。

[模型训练](#模型训练)执行训练模型的命令，需要根据自己的需求进行修改，该单元也提供一些训练参数的例子，可在例子的基础上进行修改。

如果需要快速取消注释，可以选中代码，按下`Ctrl + /`取消注释。


## 提示
1. 不同单元中包含注释, 可阅读注释获得帮助。
2. 训练代码的部分需要根据自己的需求进行更改。
3. 推荐使用 Kaggle 的 `Save Version` 的功能运行笔记，可让 Kaggle 笔记在无人值守下保持运行，直至所有单元运行完成。
4. 如果有 [HuggingFace](https://huggingface.co) 账号或者 [ModelScope](https://modelscope.cn) 账号，可通过填写 Token 和仓库名后实现自动上传训练好的模型，仓库需要手动创建。
5. 进入 Kaggle 笔记后，在 Kaggle 的右侧栏可以调整 kaggle 笔记的设置，也可以上传训练集等。注意，在 Kaggle 笔记的`Session options`->`ACCELERATOR`中，需要选择`GPU T4 x 2`，才能使用 GPU 进行模型训练。
6. 使用 Kaggle 进行模型训练时，训练集中最好没有 NSFW 内容，否则可能会导致 Kaggle 账号被封禁。
7. 不同单元的标题下方包含快捷跳转链接，可使用跳转链接翻阅 Notebook。
8. 该 Notebook 的使用方法可阅读：</br>[使用 HuggingFace / ModelScope 保存和下载文件 - licyk的小窝](https://licyk.netlify.app/2025/01/16/use-huggingface-or-modelscope-to-save-file/)</br>[使用 Kaggle 进行模型训练 - licyk的小窝](https://licyk.netlify.app/2025/01/16/use-kaggle-to-training-sd-model)


## 功能初始化
通常不需要修改该单元的内容  
1. [[下一个单元 →](#参数配置)]

In [ ]:
# SD Scripts Manager 功能初始化部分, 通常不需要修改
# 如果需要查看完整代码实现, 可阅读: https://github.com/licyk/sd-webui-all-in-one/blob/main/sd_scripts_ipynb_core.py
#################################################################################################################
# SD_SCRIPTS_IPYNB_CORE_URL, FORCE_DOWNLOAD_CORE 参数可根据需求修改, 通常保持默认即可
SD_SCRIPTS_IPYNB_CORE_URL = "https://github.com/licyk/sd-webui-all-in-one/raw/main/sd_scripts_ipynb_core.py" # SD Scripts Manager 核心下载地址
FORCE_DOWNLOAD_CORE = False # 设置为 True 时, 即使 SD Scripts Manager 已存在也会重新下载
#################################################################################################################
import os
import sys
import urllib.request
from pathlib import Path
try:
    print(f"[HDM Manager] Jupyter Notebook 根目录: {JUPYTER_ROOT_PATH}") # type: ignore
except Exception as _:
    JUPYTER_ROOT_PATH = Path(os.getcwd())
    sys.path.append(str(JUPYTER_ROOT_PATH.resolve()))
    print(f"[HDM Manager] Jupyter Notebook 根目录: {JUPYTER_ROOT_PATH}")
    SD_SCRIPTS_IPYNB_CORE_PATH = JUPYTER_ROOT_PATH / "sd_scripts_ipynb_core.py" # SD Scripts Manager 核心保存路径
try:
    if SD_SCRIPTS_IPYNB_CORE_PATH.exists() and not FORCE_DOWNLOAD_CORE:
        print("[HDM Manager] SD Scripts Manager 核心模块已存在")
    else:
        urllib.request.urlretrieve(SD_SCRIPTS_IPYNB_CORE_URL, SD_SCRIPTS_IPYNB_CORE_PATH)
        print("[HDM Manager] SD Scripts Manager 核心模块下载成功")
except Exception as e:
    raise Exception(f"SD Scripts Manager 核心模块下载错误: {e}")
from sd_scripts_ipynb_core import logger, VERSION, BaseManager
logger.info("SD Scripts Manager 版本: %s", VERSION)
logger.info("核心模块初始化完成")
#################################################################################################################


class HDMTrainManager(BaseManager):
    def install(
        self,
        torch_ver: str | list | None = None,
        xformers_ver: str | list | None = None,
        model_path: str | Path = None,
        model_list: list[str, int] | None = None,
        use_uv: bool | None = True,
        pypi_index_mirror: str | None = None,
        pypi_extra_index_mirror: str | None = None,
        pypi_find_links_mirror: str | None = None,
        github_mirror: str | list | None = None,
        huggingface_mirror: str | None = None,
        pytorch_mirror: str | None = None,
        hdm_repo: str | None = None,
        retry: int | None = 3,
        huggingface_token: str | None = None,
        modelscope_token: str | None = None,
        wandb_token: str | None = None,
        git_username: str | None = None,
        git_email: str | None = None,
        check_avaliable_gpu: bool | None = False,
        enable_tcmalloc: bool | None = True,
    ) -> None:
        """安装 HDM 和其余环境

        :param torch_ver`(str|None)`: 指定的 PyTorch 软件包包名, 并包括版本号
        :param xformers_ver`(str|None)`: 指定的 xFormers 软件包包名, 并包括版本号
        :param model_path`(str|Path|None)`: 指定模型下载的路径
        :param model_list`(list[str|int]|None)`: 模型下载列表
        :param use_uv`(bool|None)`: 使用 uv 替代 Pip 进行 Python 软件包的安装
        :param pypi_index_mirror`(str|None)`: PyPI Index 镜像源链接
        :param pypi_extra_index_mirror`(str|None)`: PyPI Extra Index 镜像源链接
        :param pypi_find_links_mirror`(str|None)`: PyPI Find Links 镜像源链接
        :param github_mirror`(str|list|None)`: Github 镜像源链接或者镜像源链接列表
        :param huggingface_mirror`(str|None)`: HuggingFace 镜像源链接
        :param pytorch_mirror`(str|None)`: PyTorch 镜像源链接
        :param hdm_repo`(str|None)`: HDM 仓库地址, 未指定时默认为`https://github.com/KohakuBlueleaf/HDM`
        :param retry`(int|None)`: 设置下载模型失败时重试次数
        :param huggingface_token`(str|None)`: 配置 HuggingFace Token
        :param modelscope_tokenn`(str|None)`: 配置 ModelScope Token
        :param wandb_token`(str|None)`: 配置 WandB Token
        :param git_username`(str|None)`: Git 用户名
        :param git_email`(str|None)`: Git 邮箱
        :param check_avaliable_gpu`(bool|None)`: 检查是否有可用的 GPU, 当 GPU 不可用时引发`Exception`
        :param enable_tcmalloc`(bool|None)`: 启用 TCMalloc 内存优化
        :notes
            self.install() 将会以下几件事
            1. 配置 PyPI / Github / HuggingFace 镜像源
            2. 配置 Pip / uv
            3. 安装管理工具自身依赖
            4. 安装 HDM
            5. 安装 PyTorch / xFormers
            6. 安装 HDM 的依赖
            7. 下载模型
            8. 配置 HuggingFace / ModelScope / WandB Token 环境变量
            9. 配置其他工具
        """
        logger.info("配置 HDM 环境中")
        os.chdir(self.workspace)
        hdm_path = self.workspace / self.workfolder
        hdm_repo = hdm_repo if hdm_repo is not None else "https://github.com/KohakuBlueleaf/HDM"
        model_path = model_path if model_path is not None else (
            self.workspace / "hdm-models")
        model_list = model_list if model_list else []
        # 检查是否有可用的 GPU
        if check_avaliable_gpu and not self.utils.check_gpu():
            raise Exception(
                "没有可用的 GPU, 请在 kaggle -> Notebook -> Session options -> ACCELERATOR 选择 GPU T4 x 2\n如果不能使用 GPU, 请检查 Kaggle 账号是否绑定了手机号或者尝试更换账号!")
        # 配置镜像源
        self.mirror.set_mirror(
            pypi_index_mirror=pypi_index_mirror,
            pypi_extra_index_mirror=pypi_extra_index_mirror,
            pypi_find_links_mirror=pypi_find_links_mirror,
            github_mirror=github_mirror,
            huggingface_mirror=huggingface_mirror
        )
        self.mirror.configure_pip()  # 配置 Pip / uv
        self.env.install_manager_depend(use_uv)  # 准备 Notebook 的运行依赖
        # 下载 HDM
        self.git.clone(
            repo=hdm_repo,
            path=hdm_path,
        )
        self.git.update(hdm_path)  # 更新 HDM
        # 安装 HDM 的依赖
        self.env.pip_install(
            "-e",
            f"{hdm_path}[finetune,tipo,liger]",
            use_uv=use_uv,
        )
        # 安装 PyTorch 和 xFormers
        self.env.install_pytorch(
            torch_package=torch_ver,
            xformers_package=xformers_ver,
            pytorch_mirror=pytorch_mirror,
            use_uv=use_uv
        )
        # 更新 urllib3
        try:
            self.env.pip_install(
                "urllib3",
                "--upgrade",
                use_uv=False
            )
        except Exception as e:
            logger.error("更新 urllib3 时发生错误: %s", e)
        try:
            self.env.pip_install(
                "numpy==1.26.4",
                use_uv=use_uv
            )
        except Exception as e:
            logger.error("降级 numpy 时发生错误: %s", e)
        self.get_model_from_list(
            path=model_path,
            model_list=model_list,
            retry=retry
        )
        self.restart_repo_manager(
            hf_token=huggingface_token,
            ms_token=modelscope_token,
        )
        self.utils.config_wandb_token(wandb_token)
        self.git.set_git_config(
            username=git_username,
            email=git_email,
        )
        enable_tcmalloc and self.utils.config_tcmalloc()
        logger.info("HDM 环境配置完成")

## 参数配置
设置必要的参数, 根据注释说明进行修改  
2. [[← 上一个单元](#功能初始化)|[下一个单元 →](#安装环境)]

In [ ]:
# 环境设置
WORKSPACE = "/kaggle" # 工作路径, 通常不需要修改
WORKFOLDER = "HDM" # 工作路径中文件夹名称, 通常不需要修改
HDM_REPO = "https://github.com/KohakuBlueleaf/HDM" # HDM 仓库地址
TORCH_VER = "" # PyTorch 版本
XFORMERS_VER = "" # xFormers 版本
USE_UV = True # 使用 uv 加速 Python 软件包安装, 修改为 True 为启用, False 为禁用
PIP_INDEX_MIRROR = "https://pypi.python.org/simple" # PyPI 主镜像源
PIP_EXTRA_INDEX_MIRROR = "https://download.pytorch.org/whl/cu128" # PyPI 扩展镜像源
PYTORCH_MIRROR = "https://download.pytorch.org/whl/cu124" # 用于下载 PyTorch 的镜像源
PIP_FIND_LINKS_MIRROR = "https://download.pytorch.org/whl/cu121/torch_stable.html" # PyPI 扩展镜像源
HUGGINGFACE_MIRROR = "https://hf-mirror.com" # HuggingFace 镜像源
GITHUB_MIRROR = [ # Github 镜像源
    "https://ghfast.top/https://github.com",
    "https://mirror.ghproxy.com/https://github.com",
    "https://ghproxy.net/https://github.com",
    "https://gh.api.99988866.xyz/https://github.com",
    "https://gh-proxy.com/https://github.com",
    "https://ghps.cc/https://github.com",
    "https://gh.idayer.com/https://github.com",
    "https://ghproxy.1888866.xyz/github.com",
    "https://slink.ltd/https://github.com",
    "https://github.boki.moe/github.com",
    "https://github.moeyy.xyz/https://github.com",
    "https://gh-proxy.net/https://github.com",
    "https://gh-proxy.ygxz.in/https://github.com",
    "https://wget.la/https://github.com",
    "https://kkgithub.com",
    "https://gitclone.com/github.com",
]
CHECK_AVALIABLE_GPU = False # 检查可用的 GPU, 当 GPU 不可用时强制终止安装进程
RETRY = 3 # 重试下载次数
DOWNLOAD_THREAD = 16 # 下载线程
ENABLE_TCMALLOC = True # 启用 TCMalloc 内存优化

##############################################################################

# 模型上传设置, 使用 HuggingFace / ModelScope 上传训练好的模型
# HuggingFace: https://huggingface.co
# ModelScope: https://modelscope.cn
USE_HF_TO_SAVE_MODEL = False # 使用 HuggingFace 保存训练好的模型, 修改为 True 为启用, False 为禁用 (True / False)
USE_MS_TO_SAVE_MODEL = False # 使用 ModelScope 保存训练好的模型, 修改为 True 为启用, False 为禁用 (True / False)

# Token 配置, 用于上传 / 下载模型 (部分模型下载需要 Token 进行验证)
# HuggingFace Token 在 Account -> Settings -> Access Tokens 中获取
HF_TOKEN = "" # HuggingFace Token
# ModelScope Token 在 首页 -> 访问令牌 -> SDK 令牌 中获取
MS_TOKEN = "" # ModelScope Token

# 用于上传模型的 HuggingFace 模型仓库的 ID, 当仓库不存在时则尝试新建一个
HF_REPO_ID = "username/reponame" # HuggingFace 仓库的 ID (格式: "用户名/仓库名")
HF_REPO_TYPE = "model" # HuggingFace 仓库的种类 (可选的类型为: model / dataset / space), 如果在 HuggingFace 新建的仓库为模型仓库则不需要修改
# HuggingFace 仓库类型和对应名称:
# model: 模型仓库
# dataset: 数据集仓库
# space: 在线运行空间仓库

# 用于上传模型的 ModelScope 模型仓库的 ID, 当仓库不存在时则尝试新建一个
MS_REPO_ID = "username/reponame" # ModelScope 仓库的 ID (格式: "用户名/仓库名")
MS_REPO_TYPE = "model" # ModelScope 仓库的种类 (model / dataset / space), 如果在 ModelScope 新建的仓库为模型仓库则不需要修改
# ModelScope 仓库类型和对应名称:
# model: 模型仓库
# dataset: 数据集仓库
# space: 创空间仓库

# 设置自动创建仓库时仓库的可见性, False 为私有仓库(不可见), True 为公有仓库(可见), 通常保持默认即可
HF_REPO_VISIBILITY = False # 设置新建的 HuggingFace 仓库可见性 (True / False)
MS_REPO_VISIBILITY = False # 设置新建的 ModelScope 仓库可见性 (True / False)

# Git 信息设置, 可以使用默认值
GIT_USER_EMAIL = "username@example.com" # Git 的邮箱
GIT_USER_NAME = "username" # Git 的用户名

##############################################################################

# 训练日志设置, 使用 WandB 记录训练日志, 使用 WandB 可远程查看实时训练日志
# WandB Token 可在 https://wandb.ai/authorize 中获取
WANDB_TOKEN = "" # WandB Token

##############################################################################

# 路径设置, 通常保持默认即可
INPUT_DATASET_PATH = "/kaggle/dataset" # 训练集保存的路径
OUTPUT_PATH = "/kaggle/working/model" # 训练时模型保存的路径
HDM_MODEL_PATH = "/kaggle/hdm-models" # 模型下载到的路径
KAGGLE_INPUT_PATH = "/kaggle/input" # Kaggle Input 的路径

##############################################################################

# 训练模型设置, 在安装时将会下载选择的模型
# 下面举个例子:
# HDM_MODEL = [
#     ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/v1-5-pruned-emaonly.safetensors", 0],
#     ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/animefull-final-pruned.safetensors", 1],
#     ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/Counterfeit-V3.0_fp16.safetensors", 0],
#     ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1.safetensors", 1, "Illustrious.safetensors"]
# ]
# 
# 在这个例子中, 第一个参数指定了模型的下载链接, 第二个参数设置了是否要下载这个模型, 当这个值为 1 时则下载该模型
# 第三个参数是可选参数, 用于指定下载到本地后的文件名称
# 
# 则上面的例子中
# https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/animefull-final-pruned.safetensors 和 
# https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1.safetensors 下载链接所指的文件将被下载
# https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/animefull-final-pruned.safetensors 的文件下载到本地后名称为 animefull-final-pruned.safetensors
# 并且 https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1.safetensors 所指的文件将被重命名为 Illustrious.safetensors

HDM_MODEL = [
    ["https://huggingface.co/KBlueLeaf/HDM-xut-340M-anime/resolve/main/hdm-xut-340M-512px-note.safetensors", 1],
    ["https://huggingface.co/KBlueLeaf/HDM-xut-340M-anime/resolve/main/hdm-xut-340M-768px-note.safetensors", 1],
    ["https://huggingface.co/KBlueLeaf/HDM-xut-340M-anime/resolve/main/hdm-xut-340M-1024px-note.safetensors", 1],
]

##############################################################################
# 下面为初始化参数部分, 不需要修改
INSTALL_PARAMS = {
    "torch_ver": TORCH_VER or None,
    "xformers_ver": XFORMERS_VER or None,
    "model_path": HDM_MODEL_PATH or None,
    "model_list": HDM_MODEL,
    "use_uv": USE_UV,
    "pypi_index_mirror": PIP_INDEX_MIRROR or None,
    "pypi_extra_index_mirror": PIP_EXTRA_INDEX_MIRROR or None,
    # Kaggle 的环境暂不需要以下镜像源
    # "pypi_find_links_mirror": PIP_FIND_LINKS_MIRROR or None,
    # "github_mirror": GITHUB_MIRROR or None,
    # "huggingface_mirror": HUGGINGFACE_MIRROR or None,
    "pytorch_mirror": PYTORCH_MIRROR or None,
    "hdm_repo": HDM_REPO or None,
    "retry": RETRY,
    "huggingface_token": HF_TOKEN or None,
    "modelscope_token": MS_TOKEN or None,
    "wandb_token": WANDB_TOKEN or None,
    "git_username": GIT_USER_NAME or None,
    "git_email": GIT_USER_EMAIL or None,
    "check_avaliable_gpu": CHECK_AVALIABLE_GPU,
    "enable_tcmalloc": ENABLE_TCMALLOC,
}
os.makedirs(WORKSPACE, exist_ok=True) # 创建工作路径
os.makedirs(OUTPUT_PATH, exist_ok=True) # 创建模型输出路径
os.makedirs(INPUT_DATASET_PATH, exist_ok=True) # 创建训练集路径
os.makedirs(HDM_MODEL_PATH, exist_ok=True) # 创建模型下载路径
HDM_PATH = os.path.join(WORKSPACE, WORKFOLDER) # HDM 路径
logger.info("参数设置完成")

## 安装环境
安装环境和下载模型和训练集, 根据注释的说明进行修改  
3. [[← 上一个单元](#参数配置)|[下一个单元 →](#模型训练)]

In [ ]:
# 初始化部分参数并执行安装命令, 这一小部分不需要修改
from tqdm import tqdm
logger.info("开始安装 HDM")
hdm_manager = HDMTrainManager(WORKSPACE, WORKFOLDER)
hdm_manager.install(**INSTALL_PARAMS)

# 将 KAGGLE_INPUT_PATH 内的文件移动到 INPUT_DATASET_PATH 指定的路径
if os.path.exists(KAGGLE_INPUT_PATH) and len(os.listdir(KAGGLE_INPUT_PATH)) > 0:
    logger.info("从 Kaggle Input 导入文件中")
    for i in tqdm(os.listdir(KAGGLE_INPUT_PATH), desc="[HDM Manager] Kaggle Input 文件导入"):
        hdm_manager.copy_files(os.path.join(KAGGLE_INPUT_PATH, i), INPUT_DATASET_PATH)
# 在 Kaggle 界面右侧中有 Kaggle Input 的功能, 可用于导入训练集 / 模型
# 如果使用了 Kaggle Input 导入了训练集 / 模型, 则 KAGGLE_INPUT_PATH 中的所有文件将被复制到 INPUT_DATASET_PATH 中
# 即将 /kaggle/input 中的所有文件复制到 /kaggle/dataset 中
##########################################################################################
# 下方可自行编写命令
# 下方的命令示例可以根据自己的需求进行修改


##### 1. 关于运行环境 #####

# 如果需要安装某个软件包, 可以使用 %pip 命令
# 下面是几个使用例子:
# 1.
# %pip install lycoris-lora==2.1.0.post3 dadaptation==3.1
# 
# 这将安装 lycoris-lora==2.1.0.post3 和 dadaptation==3.1
# 
# 2.
# %pip uninstall tensorboard
# 
# 这将卸载 tensorboard


##########################################################################################


##### 2. 关于模型导入 #####

# 该 Kaggle 训练脚本支持 4 种方式导入模型, 如下:
# 1. 使用 Kaggle Input 导入
# 2. 使用模型下载链接导入
# 3. 从 HuggingFace 仓库导入
# 4. 从 ModelScope 仓库导入


### 2.1. 使用 Kaggle Input 导入 ###
# 在 Kaggle 右侧面板中, 点击 Notebook -> Input -> Upload -> New Model, 从此处导入模型


### 2.2 使用模型下载链接导入 ###
# 如果需要通过链接下载额外的模型, 可以使用 hdm_manager.get_model()
# 使用参数:
# hdm_manager.get_model(
#     url="model_url",                    # 模型下载链接
#     path=HDM_MODEL_PATH,                 # 模型下载到本地的路径
#     filename="filename.safetensors",    # 模型的名称
#     retry=RETRY,                        # 重试下载的次数
# )
# 
# 下面是几个使用例子:
# 1.
# hdm_manager.get_model(
#     url="https://modelscope.cn/models/user/repo/resolve/master/your_model.safetensors",
#     path=HDM_MODEL_PATH,
#     retry=RETRY,
# )
# 这将从 https://modelscope.cn/models/user/repo/resolve/master/your_model.safetensors 下载模型并保存到 HDM_MODEL_PATH 中
# 
# hdm_manager.get_model(
#     url="https://modelscope.cn/models/user/repo/resolve/master/your_model.safetensors",
#     path=HDM_MODEL_PATH,
#     filename="rename_model.safetensors",
#     retry=RETRY,
# )
# 这将从 https://modelscope.cn/models/user/repo/resolve/master/your_model.safetensors 下载模型并保存到 HDM_MODEL_PATH 中, 并且重命名为 rename_model.safetensors


### 2.3. 从 HuggingFace 仓库导入 ###
# 如果需要从 HuggingFace 仓库下载模型, 可以使用 hdm_manager.repo.download_files_from_repo()
# 使用参数:
# hdm_manager.repo.download_files_from_repo(
#     api_type="huggingface",                 # 指定为 HuggingFace 的仓库
#     local_dir=HDM_MODEL_PATH,                # 模型下载到本地的路径
#     repo_id="usename/repo_id",              # HuggingFace 仓库 ID
#     repo_type="model",                      # (可选参数) HuggingFace 仓库种类 (model / dataset / space)
#     folder="path/in/repo/file.safetensors", # (可选参数) 文件在 HuggingFace 仓库中的路径
#     retry=RETRY,                            # (可选参数) 重试下载的次数, 默认为 3
#     num_threads=DOWNLOAD_THREAD,            # (可选参数) 下载线程
# )
# 
# 例如要从 stabilityai/stable-diffusion-xl-base-1.0 (类型为 model) 下载 sd_xl_base_1.0_0.9vae.safetensors
# hdm_manager.repo.download_files_from_repo(
#     api_type="huggingface",
#     repo_id="stabilityai/stable-diffusion-xl-base-1.0",
#     repo_type="model",
#     folder="sd_xl_base_1.0_0.9vae.safetensors",
#     local_dir=HDM_MODEL_PATH,
#     retry=RETRY,
#     num_threads=DOWNLOAD_THREAD,
# )
# 则上述的命令将会从 stabilityai/stable-diffusion-xl-base-1.0 下载 sd_xl_base_1.0_0.9vae.safetensors 模型
# 并将模型保存到 HDM_MODEL_PATH 中
# 注意 folder 填的是文件在 HuggingFace 仓库中的路径, 如果上述例子中的文件在仓库的 checkpoint/sd_xl_base_1.0_0.9vae.safetensors 路径
# 则 folder 填的内容为 checkpoint/sd_xl_base_1.0_0.9vae.safetensors
#
# 模型保存的路径与 local_dir 和 folder 参数有关
# 对于上面的例子 local_dir 为 /kaggle/sd-models, folder 为 sd_xl_base_1.0_0.9vae.safetensors
# 则最终保存的路径为 /kaggle/sd-models/sd_xl_base_1.0_0.9vae.safetensors
# 
# 如果 folder 为 checkpoint/sd_xl_base_1.0_0.9vae.safetensors
# 则最终保存的路径为 /kaggle/sd-models/checkpoint/sd_xl_base_1.0_0.9vae.safetensors
# 
# folder 参数为可选参数, 即该参数可不指定, 在不指定的情况下将下载整个仓库中的文件
# 比如将上面的例子改成:
# hdm_manager.repo.download_files_from_repo(
#     api_type="huggingface",
#     repo_id="stabilityai/stable-diffusion-xl-base-1.0",
#     repo_type="model",
#     local_dir=HDM_MODEL_PATH,
#     retry=RETRY,
#     num_threads=DOWNLOAD_THREAD,
# )
# 这时候将下载 stabilityai/stable-diffusion-xl-base-1.0 仓库中的所有文件
# 对于可选参数, 可以进行省略, 此时将使用该参数的默认值进行运行, 上面的例子就可以简化成下面的:
# hdm_manager.repo.download_files_from_repo(
#     api_type="huggingface",
#     repo_id="stabilityai/stable-diffusion-xl-base-1.0",
#     repo_type="model",
#     folder="sd_xl_base_1.0_0.9vae.safetensors",
#     local_dir=HDM_MODEL_PATH,
# )
# 省略后仍然可以正常执行, 但对于一些重要的可选参数, 不推荐省略, 如 repo_type 参数
# 该参数用于指定仓库类型, 不指定时则默认认为仓库为 model 类型
# 若要下载的仓库为 dataset 类型, 不指定 repo_type 参数时默认就把仓库类型当做 model, 最终导致找不到要下载的仓库


### 2.4. 从 ModelScope 仓库导入 ###
# 如果需要从 ModelScope 仓库下载模型, 可以使用 hdm_manager.repo.download_files_from_repo()
# 使用方法和 **2.3. 从 HuggingFace 仓库导入** 部分的类似, 只需要指定 api_type="modelscope" 来指定使用 ModelScope 的仓库
# 使用参数:
# hdm_manager.repo.download_files_from_repo(
#     api_type="modelscope",                  # 指定为 ModelScope 的仓库
#     local_dir=HDM_MODEL_PATH,                # 模型下载到本地的路径
#     repo_id="usename/repo_id",              # ModelScope 仓库 ID
#     repo_type="model",                      # (可选参数) ModelScope 仓库种类 (model / dataset / space)
#     folder="path/in/repo/file.safetensors", # (可选参数) 文件在 ModelScope 仓库中的路径
#     retry=RETRY,                            # (可选参数) 重试下载的次数, 默认为 3
#     num_threads=DOWNLOAD_THREAD,            # (可选参数) 下载线程
# )
# 
# 例如要从 stabilityai/stable-diffusion-xl-base-1.0 (类型为 model) 下载 sd_xl_base_1.0_0.9vae.safetensors
# hdm_manager.dataset.get_single_file_from_ms(
#     api_type="modelscope",
#     repo_id="stabilityai/stable-diffusion-xl-base-1.0",
#     repo_type="model",
#     folder="sd_xl_base_1.0_0.9vae.safetensors",
#     local_dir=HDM_MODEL_PATH,
#     retry=RETRY,
#     num_threads=DOWNLOAD_THREAD,
# )
# 则上述的命令将会从 stabilityai/stable-diffusion-xl-base-1.0 下载 sd_xl_base_1.0_0.9vae.safetensors 模型
# 并将模型保存到 HDM_MODEL_PATH 中



##########################################################################################


##### 3. 关于训练集导入 #####

# 该 Kaggle 训练脚本支持 4 种方式导入训练集, 如下:
# 1. 使用 Kaggle Input 导入
# 2. 使用训练集下载链接导入
# 3. 从 HuggingFace 仓库导入
# 4. 从 ModelScope 仓库导入


### 3.1. 使用 Kaggle Input 导入 ###
# 在 Kaggle 右侧面板中, 点击 Notebook -> Input -> Upload -> New Dataset, 从此处导入模型


### 3.2. 使用训练集下载链接导入 ###
# 如果将训练集压缩后保存在某个平台, 如 HuggingFace, ModelScope, 并且有下载链接
# 可以使用 hdm_manager.utils.download_archive_and_unpack() 函数下载训练集
# 使用参数:
# hdm_manager.utils.download_archive_and_unpack(
#     url="download_url",             # 训练集压缩包的下载链接
#     local_dir=INPUT_DATASET_PATH,   # 下载数据集到本地的路径
#     name="filename.zip",            # (可选参数) 将数据集压缩包进行重命名
#     retry=RETRY,                    # (可选参数) 重试下载的次数
# )
# 
# 该函数在下载训练集压缩包完成后将解压到指定的本地路径
# 压缩包格式仅支持 7z, zip, tar
# 
# 下面是几个使用的例子:
# 1.
# hdm_manager.utils.download_archive_and_unpack(
#     url="https://modelscope.cn/models/user/repo/resolve/master/data_1.7z",
#     local_dir=INPUT_DATASET_PATH,
#     retry=RETRY,
# )
# 这将从 https://modelscope.cn/models/user/repo/resolve/master/data_1.7z 下载训练集压缩包并解压到 INPUT_DATASET_PATH 中
# 
# 2.
# hdm_manager.utils.download_archive_and_unpack(
#     url="https://modelscope.cn/models/user/repo/resolve/master/data_1.7z",
#     local_dir=INPUT_DATASET_PATH,
#     name="training_dataset.7z",
#     retry=RETRY,
# )
# 这将从 https://modelscope.cn/models/user/repo/resolve/master/data_1.7z 下载训练集压缩包并重命名成 training_dataset.7z
# 再将 training_dataset.7z 中的文件解压到 INPUT_DATASET_PATH 中
# 
# 
# 训练集的要求:
# 需要将图片进行打标, 并调整训练集为指定的目结构, 例如:
# Nachoneko
#     └── 1_nachoneko
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.png
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.txt
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).png
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).txt
#             ├── 0(8).txt
#             ├── 0(8).webp
#             ├── 001_2.png
#             ├── 001_2.txt
#             ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.png
#             ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.txt
#             ├── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.png
#             └── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.txt
# 
# 在 Nachoneko 文件夹新建一个文件夹, 格式为 <数字>_<名称>, 如 1_nachoneko, 前面的数字代表这部分的训练集的重复次数, 1_nachoneko 文件夹内则放图片和打标文件
# 
# 训练集也可以分成多个部分组成, 例如:
# Nachoneko
#     ├── 1_nachoneko
#     │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.png
#     │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.txt
#     │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).png
#     │       └── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).txt
#     ├── 2_nachoneko
#     │       ├── 0(8).txt
#     │       ├── 0(8).webp
#     │       ├── 001_2.png
#     │       └── 001_2.txt
#     └── 4_nachoneko
#             ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.png
#             ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.txt
#             ├── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.png
#             └── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.txt
# 
# 处理好训练集并调整好目录结构后可以将 Nachoneko 文件夹进行压缩了, 使用 zip / 7z / tar 格式进行压缩
# 例如将上述的训练集压缩成 Nachoneko.7z, 此时需要检查一下压缩后在压缩包的目录结果是否和原来的一致(有些压缩软件在部分情况下会破坏原来的目录结构)
# 确认没有问题后将该训练集上传到网盘, 推荐使用 HuggingFace / ModelScope


### 3.3. 从 HuggingFace 仓库导入 ###
# 如果训练集保存在 HuggingFace, 可以使用 hdm_manager.repo.download_files_from_repo() 函数从 HuggingFace 下载数据集
# 使用方法和 **2.3. 从 HuggingFace 仓库导入** 部分类似, 部分说明可参考那部分的内容
# 使用格式:
# hdm_manager.repo.download_files_from_repo(
#     api_type="huggingface",                 # 指定为 HuggingFace 的仓库
#     local_dir=INPUT_DATASET_PATH,           # 下载数据集到哪个路径
#     repo_id="username/train_data",          # HuggingFace 仓库 ID
#     repo_type="dataset",                    # (可选参数) HuggingFace 仓库的类型 (model / dataset / space)
#     folder="folder_in_repo",                # (可选参数) 指定要从 HuggingFace 仓库里下载哪个文件夹的内容
#     retry=RETRY,                            # (可选参数) 重试下载的次数, 默认为 3
#     num_threads=DOWNLOAD_THREAD,            # (可选参数) 下载线程
# )
# 
# 比如在 HuggingFace 的仓库为 username/train_data, 仓库类型为 dataset
# 仓库的文件结构如下:
# ├── Nachoneko
# │   ├── 1_nachoneko
# │   │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.png
# │   │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.txt
# │   │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).png
# │   │       └── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).txt
# │   ├── 2_nachoneko
# │   │       ├── 0(8).txt
# │   │       ├── 0(8).webp
# │   │       ├── 001_2.png
# │   │       └── 001_2.txt
# │   └── 4_nachoneko
# │           ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.png
# │           ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.txt
# │           ├── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.png
# │           └── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.txt
# └ aaaki
#   ├── 1_aaaki
#   │   ├── 1.png
#   │   ├── 1.txt
#   │   ├── 11.png
#   │   ├── 11.txt
#   │   ├── 12.png
#   │   └── 12.txt
#   └── 3_aaaki
#       ├── 14.png
#       ├── 14.txt
#       ├── 16.png
#       └── 16.txt
#
# 此时想要下载这个仓库中的 Nachoneko 文件夹的内容, 则下载命令为
# hdm_manager.repo.download_files_from_repo(
#     api_type="huggingface",
#     local_dir=INPUT_DATASET_PATH,
#     repo_id="username/train_data",
#     repo_type="dataset",
#     folder="Nachoneko",
#     retry=RETRY,
#     num_threads=DOWNLOAD_THREAD,
# )
# 
# 如果想下载整个仓库, 则移除 folder 参数, 命令修改为
# hdm_manager.repo.download_files_from_repo(
#     api_type="huggingface",
#     local_dir=INPUT_DATASET_PATH,
#     repo_id="username/train_data",
#     repo_type="dataset",
#     retry=RETRY,
#     num_threads=DOWNLOAD_THREAD,
# )



# 4. 从 ModelScope 仓库导入
# 如果训练集保存在 ModelScope, 可以使用 hdm_manager.repo.download_files_from_repo() 函数从 ModelScope 下载数据集
# 使用方法可参考 **3.2. 使用训练集下载链接导入** 部分的说明
# 使用格式:
# hdm_manager.repo.download_files_from_repo(
#     api_type="modelscope",          # 指定为 ModelScope 的仓库
#     local_dir=INPUT_DATASET_PATH,   # 下载数据集到哪个路径
#     repo_id="usename/repo_id",      # ModelScope 仓库 ID
#     repo_type="dataset",            # (可选参数) ModelScope 仓库的类型 (model / dataset / space)
#     folder="folder_in_repo",        # (可选参数) 指定要从 ModelScope 仓库里下载哪个文件夹的内容
#     retry=RETRY,                    # (可选参数) 重试下载的次数, 默认为 3
#     num_threads=DOWNLOAD_THREAD,    # (可选参数) 下载线程
# )
# 
# 比如在 ModelScope 的仓库为 username/train_data, 仓库类型为 dataset
# 仓库的文件结构如下:
# ├── Nachoneko
# │   ├── 1_nachoneko
# │   │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.png
# │   │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.txt
# │   │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).png
# │   │       └── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).txt
# │   ├── 2_nachoneko
# │   │       ├── 0(8).txt
# │   │       ├── 0(8).webp
# │   │       ├── 001_2.png
# │   │       └── 001_2.txt
# │   └── 4_nachoneko
# │           ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.png
# │           ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.txt
# │           ├── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.png
# │           └── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.txt
# └ aaaki
#   ├── 1_aaaki
#   │   ├── 1.png
#   │   ├── 1.txt
#   │   ├── 11.png
#   │   ├── 11.txt
#   │   ├── 12.png
#   │   └── 12.txt
#   └── 3_aaaki
#       ├── 14.png
#       ├── 14.txt
#       ├── 16.png
#       └── 16.txt
#
# 此时想要下载这个仓库中的 Nachoneko 文件夹的内容, 则下载命令为
# hdm_manager.repo.download_files_from_repo(
#     api_type="modelscope",
#     local_dir=INPUT_DATASET_PATH,
#     repo_id="username/train_data",
#     repo_type="dataset",
#     folder="Nachoneko",
# )
# 
# 如果想下载整个仓库, 则移除 folder 参数, 命令修改为
# hdm_manager.repo.download_files_from_repo(
#     api_type="modelscope",
#     local_dir=INPUT_DATASET_PATH,
#     repo_id="username/train_data",
#     repo_type="dataset",
# )



# 下载训练集的技巧
# 如果有个 character_aaaki 训练集上传到 HuggingFace 上时结构如下：
# 
#
# HuggingFace_Repo (licyk/sd_training_dataset)
# ├── character_aaaki
# │   ├── 1_aaaki
# │   │   ├── 1.png
# │   │   ├── 1.txt
# │   │   ├── 3.png
# │   │   └── 3.txt
# │   └── 2_aaaki
# │       ├── 4.png
# │       └── 4.txt
# ├── character_robin
# │   └── 1_xxx
# │       ├── 11.png
# │       └── 11.txt
# └── style_pvc
#     └── 5_aaa
#         ├── test.png
#         └── test.txt
#
# 
# 可能有时候不想为训练集中每个子训练集设置不同的重复次数，又不想上传的时候再多套一层文件夹，就把训练集结构调整成了下面的：
# 
#
# HuggingFace_Repo (licyk/sd_training_dataset)
# ├── character_aaaki
# │   ├── 1.png
# │   ├── 1.txt
# │   ├── 3.png
# │   ├── 3.txt
# │   ├── 4.png
# │   └── 4.txt
# ├── character_robin
# │   └── 1_xxx
# │       ├── 11.png
# │       └── 11.txt
# └── style_pvc
#     └── 5_aaa
#         ├── test.png
#         └── test.txt
#
# 
# 此时这个状态的训练集是缺少子训练集和重复次数的，如果直接使用 hdm_manager.repo.download_files_from_repo() 去下载训练集并用于训练将会导致报错
# 不过可以自己再编写一个函数对 hdm_manager.repo.download_files_from_repo() 函数再次封装，自动加上子训练集并设置重复次数
# 
#
# def make_dataset(
#     local_dir: str | Path,
#     repo_id: str,
#     repo_type: str,
#     repeat: int,
#     folder: str,
# ) -> None:
#     import os
#     import shutil
#     origin_dataset_path = os.path.join(local_dir, folder)
#     tmp_dataset_path = os.path.join(local_dir, f"{repeat}_{folder}")
#     new_dataset_path = os.path.join(origin_dataset_path, f"{repeat}_{folder}")
#     hdm_manager.repo.download_files_from_repo(
#         api_type="huggingface",
#         local_dir=local_dir,
#         repo_id=repo_id,
#         repo_type=repo_type,
#         folder=folder,
#     )
#     if os.path.exists(origin_dataset_path):
#         logger.info("设置 %s 训练集的重复次数为 %s", folder, repeat)
#         shutil.move(origin_dataset_path, tmp_dataset_path)
#         shutil.move(tmp_dataset_path, new_dataset_path)
#     else:
#         logger.error("从 %s 下载 %s 失败", repo_id, folder)
#
# 
# 编写好后，可以去调用这个函数
# 
#
# make_dataset(
#     local_dir=INPUT_DATASET_PATH,
#     repo_id="licyk/sd_training_dataset",
#     repo_type="dataset",
#     repeat=3,
#     folder="character_aaaki",
# )
#
# 
# 该函数将会把 character_aaaki 训练集下载到 {INPUT_DATASET_PATH} 中，即 /kaggle/dataset
# 文件夹名称为 character_aaaki，并且 character_aaaki 文件夹内继续创建了一个子文件夹作为子训练集，根据 repeat=3 将子训练集的重复次数设置为 3


##########################################################################################
logger.info("HDM 安装完成")

## 模型训练
需自行编写命令，下方有可参考的例子  
4. [[← 上一个单元](#安装环境)|[下一个单元 →](#模型上传)]

In [ ]:
logger.info("进入 HDM 目录")
os.chdir(HDM_PATH)
print("=" * 50)
logger.info("训练集目录中的文件列表")
if os.path.exists(INPUT_DATASET_PATH):
    print(f"训练集路径: {INPUT_DATASET_PATH}")
    for i in os.listdir(INPUT_DATASET_PATH):
        print(f":: {i}")
print("=" * 50)
logger.info("模型目录中的文件列表")
if os.path.exists(HDM_MODEL_PATH):
    print(f"模型路径: {HDM_MODEL_PATH}")
    for i in os.listdir(HDM_MODEL_PATH):
        print(f":: {i}")
print("=" * 50)
logger.info("使用 HDM 进行模型训练")
##########################################################################################
# 1.
# 运行前需要根据自己的需求更改参数
# 
# 训练参数的设置可参考：
# https://github.com/KohakuBlueleaf/HDM/blob/main/config/train/hdm-xut-340M-ft.toml
# 
# 
# 2.
# 下方被注释的代码选择后使用 Ctrl + / 取消注释
# 
# 
# 3.
# 训练使用的底模会被下载到 HDM_MODEL_PATH, 即 /kaggle/sd-models
# 填写底模路径时一般可以通过 --pretrained_model_name_or_path="{HDM_MODEL_PATH}/base_model.safetensors" 指定
# 如果需要外挂 VAE 模型可以通过 --vae="{HDM_MODEL_PATH}/vae.safetensors" 指定
# 
# 通过 Kaggle Inout 导入的训练集保存在 KAGGLE_INPUT_PATH, 即 /kaggle/input, 运行该笔记时将会把训练集复制进 INPUT_DATASET_PATH, 即 /kaggle/dataset
# 该路径可通过 INPUT_DATASET_PATH 调整
# 如果使用 hdm_manager.dataset.get_dataset() 函数下载训练集, 数据集一般会解压到 INPUT_DATASET_PATH, 这取决于函数第一个参数传入的路径
# 训练集的路径通常要这种结构
# $ tree /kaggle
# kaggle
# └── dataset
#     └── Nachoneko
#         └── 1_gan_cheng
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.png
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.txt
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).png
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).txt
#             ├── 0(8).txt
#             ├── 0(8).webp
#             ├── 001_2.png
#             ├── 001_2.txt
#             ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.png
#             ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.txt
#             ├── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.png
#             └── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.txt
# 4 directories, 12 files
# 在填写训练集路径时, 应使用 --train_data_dir="{INPUT_DATASET_PATH}/Nachoneko"
# 
# 模型保存的路径通常用 --output_dir="{OUTPUT_PATH}" 指定, 如 --output_dir="{OUTPUT_PATH}/Nachoneko", OUTPUT_PATH 默认设置为 /kaggle/working/model
# 在 Kaggle 的 Output 中可以看到保存的模型, 前提是使用 Kaggle 的 Save Version 运行 Kaggle
# OUTPUT_PATH 也指定了保存模型到 HuggingFace / ModelScope 的功能的上传路径
# 
# --output_name 用于指定保存的模型名字, 如 --output_name="Nachoneko"
# 
# 
# 4.
# Kaggle 的实例最长可运行 12 h, 要注意训练时长不要超过 12 h, 否则将导致训练被意外中断, 并且最后的模型保存功能将不会得到运行
# 如果需要在模型被保存后立即上传到 HuggingFace 进行保存, 可使用启动参数为 sd-scripts 设置自动保存, 具体可阅读 sd-scripts 的帮助信息
# 使用 python train_network.py -h 命令可查询可使用的启动参数, 命令中的 train_network.py 可替换成 sdxl_train_network.py 等
# 
# 
# 5.
# 训练命令的开头为英文的感叹号, 也就是 !, 后面就是 Shell Script 风格的命令
# 每行的最后为反斜杠用于换行, 也就是用 \ 来换行, 并且反斜杠的后面不允许有其他符号, 比如空格等
# 训练命令的每一行之间不能有任何换行空出来, 最后一行不需要反斜杠, 因为最后一行的下一行已经没有训练参数
# 
# 
# 6.
# 如果训练参数是 toml 格式的, 比如从 Akegarasu/lora-scripts 训练器复制来的训练参数
# 可以转换成对应的训练命令中的参数
# 下面列举几种转换例子:
# 
# (1)
# toml 格式:
# pretrained_model_name_or_path = "{HDM_MODEL_PATH}/Illustrious-XL-v0.1.safetensors"
# 训练命令格式:
# --pretrained_model_name_or_path="{HDM_MODEL_PATH}/Illustrious-XL-v0.1.safetensors"
# 
# (2)
# toml 格式:
# unet_lr = 0.0001
# 训练命令格式:
# --unet_lr=0.0001
# 
# (3)
# toml 格式:
# network_args = [
#     "conv_dim=100000",
#     "conv_alpha=100000",
#     "algo=lokr",
#     "dropout=0",
#     "factor=8",
#     "train_norm=True",
#     "preset=full",
# ]
# 训练命令格式:
# --network_args \
#     conv_dim=100000 \
#     conv_alpha=100000 \
#     algo=lokr \
#     dropout=0 \
#     factor=8 \
#     train_norm=True \
#     preset=full \
# 
# (4)
# toml 格式:
# enable_bucket = true
# 训练命令格式:
# --enable_bucket
# 
# (5)
# toml 格式:
# lowram = false
# 训练命令格式:
# 无对应的训练命令, 也就是不需要填, 因为这个参数的值为 false, 也就是无对应的参数, 如果值为 true, 则对应训练命令中的 --lowram
# 
# 可以根据这个例子去转换 toml 格式的训练参数成训练命令的格式
# 
# 
# 7.
# 如果需要 toml 格式的配置文件来配置训练参数可以使用下面的代码来保存 toml 格式的训练参数
# 
# toml_file_path = os.path.join(WORKSPACE, "train_config.toml")
# toml_content = f"""
# 这里使用 toml 格式编写训练参数, 
# 还可以结合 Python F-Strings 的用法使用前面配置好的变量
# Python F-Strings 的说明: https://docs.python.org/zh-cn/3.13/reference/lexical_analysis.html#f-strings
# toml 的语法可参考: https://toml.io/cn/v1.0.0
# 下面展示训练命令里参数对应的 toml 格式转换
# 
# 
# pretrained_model_name_or_path = "{HDM_MODEL_PATH}/Illustrious-XL-v0.1.safetensors"
# 对应训练命令中的 --pretrained_model_name_or_path="{HDM_MODEL_PATH}/Illustrious-XL-v0.1.safetensors"
# 
# unet_lr = 0.0001
# 对应训练命令中的 --unet_lr=0.0001
# 
# network_args = [
#     "conv_dim=100000",
#     "conv_alpha=100000",
#     "algo=lokr",
#     "dropout=0",
#     "factor=8",
#     "train_norm=True",
#     "preset=full",
# ]
# 对应下面训练命令中的
# --network_args \
#     conv_dim=100000 \
#     conv_alpha=100000 \
#     algo=lokr \
#     dropout=0 \
#     factor=8 \
#     train_norm=True \
#     preset=full \
# 
# enable_bucket = true
# 对应训练命令中的 --enable_bucket
# 
# lowram = false
# 这个参数的值为 false, 也就是无对应的参数, 如果值为 true, 则对应训练命令中的 --lowram
# """.strip()
# if not os.path.exists(os.path.dirname(toml_file_path)):
#     os.makedirs(toml_file_path, exist_ok=True)
# with open(toml_file_path, "w", encoding="utf8") as file:
#     file.write(toml_content)
# 
# 使用上面的代码将会把训练参数的 toml 配置文件保存在 toml_file_path 路径中, 也就是 {WORKSPACE}/train_config.toml, 即 /kaggle/train_config.toml
# 而原来的训练命令无需再写上训练参数, 只需指定该训练配置文件的路径即可
# 使用 --config_file="{WORKSPACE}/train_config.toml" 来指定
# 
# 
# 8. 
# 如果要查看 sd-script 的命令行参数, 可以加上 -h 后再运行, 此时 sd-script 将显示所有可用的参数
# 
# 
# 9.
# 下方提供了一些训练参数, 可以直接使用, 使用时取消注释后根据需求修改部分参数即可
# 
#              .,@@@@@@@@@].                                          ./@[`....`[\\.                 
#             //\`..  . ...,\@].       .,]]]/O@@@@@@@@\]...       .,]//............\@`               
#           .O`........ .......\\.]]@@@@@@@@[..........,[@@@@\`.*/....=^............/@@`             
#          .O........    .......@@/@@@/`.....               . ,\@\....\`............O@`@             
#          =^...`....          .O@@`.........            .........\@`...[`.,@`....,@^/.@^            
#         .OO`..\....          =/..... ......            ..[@]....,\@@]]]].@@]`..//..@=\^            
#          @O/@`,............=O/......    ...   ....       ...\\.....,@@@`=\@\@@[...=O`/^.           
#          @@\.,@]..]]//[,/@^O=@.............   .\@^...........,@`.....\@@/*\o*O@\.=/.@`             
#          ,@/O`...[OOO`.,@O,\/....././\^....   ..@O` ..\`.......=\.....=\\@@@@@/\@@//               
#            ,@`\].......O^o,/.....@`/=^.....,\...,@^ ...=\...    =\.....,@,@@@@[/@@@/               
#            ..,\@\]]]]O/@.*@.....=^/\^......=@....\O..^..@@`..  ..\@.....,@.\@@\[[O`                
#                .*=@@\@@^.O^...../o^O.......O=^...=@..@..\.\\.   . @@`....,@.\@@@@`                 
# .              ..=@O^=@`,@ .....@@=`......=^.O....@..@^.=^.=@.....=@@.....,\.\@@@@.                
#                .,@@`,O@./^.....=@O/......./^.O... \`.=^.=^..=@...  O=\.....=^.\@@@@`.              
#                ./@`.=^@=@......=@O`....@,/@@.=^...=^.=^.=^.[[\@....=^\^.....@@.\@@@@`              
#               .,@^. @^O/@......=@O.]O`OO.=`\^.....,^.=@.=^....=@...=\.O.....@^\`@@/`               
#                =@ .=@..@^ .....=@/.../@^,/.=@......* =@.=^.....=\..=@`=^....=^ \/\                 
#                /^..=@.,@^ /`...=@.../O@.O...@........O=^=` ,`...@^.=@\=^..] =@..@O`.               
#               ,@...@/.=@. @^...=@../@\@/OOO.=^......=^,O@[.]]@\]/@ =^@`O..O.=@^ =@^                
#               =^...@@.=@..O\....@ //.O@O@@]..@....../^.OO@@@[[@@@@\/^@^O .O.=@@ .@^                
#               @^..=@@.,@.=^@....@@\@@@[[[[[[[\^@^..,/..O..,@@@\..=@@//OO..O./^@. =@                
#               @...=^@^.@.=^=^...=@@`/@@@@@`...*O\..@...[.=@`,@@@`.@`=^@=`.O.@.@. =@.               
#               @^..O.@^ \^=@,\..=@@ @\,@@/@@`..=^..@`.....@@\@@/@@...O.@=^,O=^.@^./@                
#               @@..O.=@.,\=@^O..=`\/@^/@@OO@^..,`,O`.. .. @@/@@\@@..=`=@../^O..@^/O^                
#              .=@^ @..@`=@o@@=^..,.O@@/@@oO@........... ...@^.\/@..=^=@/ =@O. .@\@`.                
#               .@@.@^.@^@^\@@O^..=^,O@@*.................  .......=^/@@^=@@^ .=@`.                  
#               .=O@O^,@^=^.\@^o..=/\,\ .....   .... .....    ...]@O`=@O/@@^   =@                    
#                .=O@O/^==@@`O@O^.=@.\`\`....      .  ........ ......//@.@`.   =^                    
#                  ,O@@^.O..\@@@^.=@...[O@`..      .  ........ .....//.@@,\   .@^                    
#                   .@/@@@]../@@^..@@\........     ..,/`=@/@`.....,@^..=^ =` .=@.                    
#                   =/..O... @^=\. O@@@@\....... ...//.,@..@O].,/@`=\..=@..@..@^                     
#                  ,/..O....=/=/@ .=@@@@@@@@].....//.,O@`.//.@@@@@..@^..@`.=\/O`.                    
#                 ,@../`....O=/.@...@@@@@@@@@@@@@/../\@..//.=@@@@\. @@..=\..@^/.                     
#                ,@`.=`....=@/..@...\@@@@@@@@.. O..,@/..=@ .O=@@@^. @/@..@^..@`.                     
#               ,@`.=^....,@/..=O^..,@@.[@@@@,@]@../^..=@../^=@@@...@.,\.=@`..@`                     
#              ,@..=/.   .@/...O=@...@@....,@@...,[@\.,@`..@..@@/..,@..,\.@@...@..                   
#              @`.,/.....@/...=`O@^..,@...../`.......,\@]..@..O@\]]/\`..,\,@^..,@`                   
#             =^..@...../@...,^/O@@...=@`...@............\@` /`,\,@`.=\.,\@=@`..,@.                  
#            ,@../`....@\`/@``,`]/@^...O,\/\@]..............\\..=@`\\ ,@@@@@@@....@`.                
#            O`.=^....@^O@^.@@@@@@@\....\@^=@@@@@\] ..........,@`\@\.@`=@@@@@@\....@`..              
#           =/..O...,@O/@^.@@@@@@@@@`...=/.@@@@@@@@@@@].........,@@/@`,@/@@@@O\^....@`..             
#           /^.O.../@O^@^./@@@@@@@@@\...@`=@@@@@@@@@@@@@\.......@`=\//@`,@@@@@.@`....\`...           
#         .,@.=^../`@@\@.=@@@@@@@@@@@^.=@.@@@@@@@@@@@@@@@@@`...@` =\\==`@`\@@@^.@.....\^...          
#       ../\^,@.,/.=@/=/,@@@@@@@@@@@@\ @^=@@@@@@@@@@@@@@@@@@@]@@@@@@@@@o\@`.@@\.,@.....\\...         
#       =/.@^@`/`..@@^@^=@@@@@@@@@@@@@\@.@@@@@@@@@@@@@@@@@@@@@@@@O@@@@@\/.,/.@@\.,@.....,@`..        
#     ,O`.,@=@/...=@@.@^O@@@@@@@@@@@@@@^=@@@@@@@@@@@@@@@@@@@@@@@@@@O@@@^ /@@.,@/@`.@`.....\\...      
#  ..=^...=^@^....@OO,@^/@@@@@@@@@@@@\@.@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\@@@@@`=\.\^.\`.....,@`..    
# 
# 炼丹就不存在什么万能参数的, 可能下面给的参数里有的参数训练出来的效果很好, 有的效果就一般
# 训练参数还看训练集呢, 同一套参数在不同训练集上效果都不一样, 可能好可能坏 (唉, 被这训练参数折磨过好多次)
# 虽然说一直用同个效果不错的参数可能不会出现特别坏的结果吧
# 还有好的训练集远比好的训练参数更重要
# 好的训练集真的, 真的, 真的非常重要
# 再好的参数, 训练集烂也救不回来
# 
# 
# 10.
# 建议先改改训练集路径的参数就开始训练, 跑通训练了再试着改其他参数
# 还有我编写的训练参数不一定是最好的, 所以需要自己去摸索这些训练参数是什么作用的, 再去修改
# 其实有些参数我自己也调不明白, 但是很多时候跑出来效果还不错
# 为什么效果好, 分からない, 这东西像个黑盒, 有时候就觉得神奇呢
##########################################################################################


# 根据 https://github.com/KohakuBlueleaf/HDM/blob/main/config/train/hdm-xut-340M-ft.toml 修改的训练参数
# 
# toml_file_path = os.path.join(WORKSPACE, "train_config.toml")
# toml_content = f"""
# [lightning]
#     seed=20090220
#     epochs=10
#     batch_size=8
#     dataloader_workers=4
#     persistent_workers=true
#     grad_acc=4
#     devices=1
#     precision="16-mixed"
#     grad_clip=1.0
#     grad_ckpt=true
# 
#     [lightning.imggencallback]
#         id="HDM-xut-340M-finetune"
#         size=1024
#         num=32
#         preview_num=8
#         batch_size=4
#         steps=32
#         period=128
#     [lightning.logger]
#         name="HDM-xut-340M-finetune"
#         project="HDM"
#         offline=true
# 
# 
# [trainer]
#     name="test"
#     lr=0.1 # We have muP scale, need higher LR here
#     optimizer="torch.optim.AdamW"
#     opt_configs = {{"weight_decay"= 0.01, "betas"= [0.9, 0.95]}}
#     lr_sch_configs = {{"end"= -1, "mode"= "cosine", "warmup"= 1000, "min_value"= 0.01}}
#     te_use_normed_ctx=false
# 
# 
# [dataset]
#     [[dataset.datasets]]
#         class = "hdm.data.kohya.KohyaDataset"
#         [dataset.datasets.kwargs]
#             size=1024
#             dataset_folder = "{INPUT_DATASET_PATH}/test_dataset"
#             keep_token_seperator="|||"
#             tag_seperator=", "
#             seperator=", "
#             group_seperator="%%"
#             tag_shuffle=true
#             group_shuffle=false
#             tag_dropout_rate=0.0
#             group_dropout_rate=0.0
#             use_cached_meta=true
#             # For example:
#             # "xxx, zzz ||| aa $$ bb %% cc $$ dd" -> "xxx, zzz, aa, bb, dd, cc"
# 
# 
# [model]
#     config="{HDM_PATH}/config/model/xut-qwen3-sm-tread.yaml"
#     model_path="{HDM_MODEL_PATH}/hdm-xut-340M-1024px-note.safetensors"
#     inference_dtype = "torch.float16"
#     [model.lycoris]
#         algo = "lokr"
#         factor = 4
#         full_matrix = true
#         train_norm = true
# 
# """.strip()
# if not os.path.exists(os.path.dirname(toml_file_path)):
#     os.makedirs(toml_file_path, exist_ok=True)
# with open(toml_file_path, "w", encoding="utf8") as file:
#     file.write(toml_content)
# !python "{HDM_PATH}/scripts/train.py" "{toml_file_path}"


# 上面那个官方示例参数会报错, 不懂为什么, 这个是随便改过的, 倒是能跑
# 
# toml_file_path = os.path.join(WORKSPACE, "train_config.toml")
# toml_content = f"""
# [lightning]
#     seed=20090220
#     epochs=10
#     batch_size=8
#     dataloader_workers=4
#     persistent_workers=true
#     grad_acc=4
#     devices=1
#     precision="16-mixed"
#     grad_clip=1.0
#     grad_ckpt=true
# 
#     [lightning.imggencallback]
#         id="HDM-xut-340M-finetune"
#         size=1024
#         num=32
#         preview_num=8
#         batch_size=4
#         steps=32
#         period=128
#     [lightning.logger]
#         name="HDM-xut-340M-finetune"
#         project="HDM"
#         offline=true
# 
# 
# [trainer]
#     name="test"
#     lr=0.1 # We have muP scale, need higher LR here
#     optimizer="torch.optim.AdamW"
#     opt_configs = {{"weight_decay"= 0.01, "betas"= [0.9, 0.95]}}
#     lr_sch_configs = {{"mode"= "cosine", "warmup"= 1000, "min_value"= 0.01}}
#     te_use_normed_ctx=false
# 
# 
# [dataset]
#     [[dataset.datasets]]
#         class = "hdm.data.kohya.KohyaDataset"
#         [dataset.datasets.kwargs]
#             size=1024
#             dataset_folder = "{INPUT_DATASET_PATH}/test_dataset"
#             keep_token_seperator="|||"
#             tag_seperator=", "
#             seperator=", "
#             group_seperator="%%"
#             tag_shuffle=true
#             group_shuffle=false
#             tag_dropout_rate=0.0
#             group_dropout_rate=0.0
#             use_cached_meta=true
#             # For example:
#             # "xxx, zzz ||| aa $$ bb %% cc $$ dd" -> "xxx, zzz, aa, bb, dd, cc"
# 
# 
# [model]
#     config="{HDM_PATH}/config/model/xut-qwen3-sm-tread.yaml"
#     model_path="{HDM_MODEL_PATH}/hdm-xut-340M-1024px-note.safetensors"
#     inference_dtype = "torch.float16"
#     [model.lycoris]
#         algo = "lokr"
#         factor = 4
#         full_matrix = true
#         train_norm = true
# 
# """.strip()
# if not os.path.exists(os.path.dirname(toml_file_path)):
#     os.makedirs(toml_file_path, exist_ok=True)
# with open(toml_file_path, "w", encoding="utf8") as file:
#     file.write(toml_content)
# !python "{HDM_PATH}/scripts/train.py" "{toml_file_path}"


##########################################################################################
os.chdir(WORKSPACE)
logger.info("离开 HDM 目录")
logger.info("模型训练结束")

## 模型上传
通常不需要修改该单元内容，如果需要修改参数，建议通过上方的参数配置单元进行修改  
5. [← 上一个单元](#模型训练)

In [ ]:
# 模型上传到 HuggingFace / ModelScope, 通常不需要修改, 修改参数建议通过上方的参数配置单元进行修改

# 使用 HuggingFace 上传模型
if USE_HF_TO_SAVE_MODEL:
    logger.info("使用 HuggingFace 保存模型")
    hdm_manager.repo.upload_files_to_repo(
        api_type="huggingface",         # 指定使用 HuggingFace 仓库
        repo_id=HF_REPO_ID,             # HuggingFace 仓库地址
        repo_type=HF_REPO_TYPE,         # HuggingFace 仓库种类
        visibility=HF_REPO_VISIBILITY,  # 当尝试创建仓库时设置仓库的可见性
        upload_path=OUTPUT_PATH,        # 要上传文件的目录
        retry=RETRY,                    # 重试上传的次数
    )

# 使用 ModelScope 上传模型
if USE_MS_TO_SAVE_MODEL:
    logger.info("使用 ModelScope 保存模型")
    hdm_manager.repo.upload_files_to_repo(
        api_type="modelscope",          # 指定使用 ModelScope 仓库
        repo_id=MS_REPO_ID,             # Modelscope 的仓库地址
        repo_type=MS_REPO_TYPE,         # ModelScope 仓库的种类
        visibility=MS_REPO_VISIBILITY,  # 当尝试创建仓库时设置仓库的可见性
        upload_path=OUTPUT_PATH,        # 要上传文件的目录
        retry=RETRY,                    # 重试上传的次数
    )